<a href="https://colab.research.google.com/github/mjalalimanesh/Statistical-Learning-ISLR-Python/blob/master/lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3.6  Lab: Linear Regression

### 3.6.1 Libraries

In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.api as sm
import statsmodels.formula.api as smf

plt.style.use('seaborn') # pretty matplotlib plots

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Load Data

In [8]:
path = "https://raw.githubusercontent.com/mjalalimanesh/Statistical-Learning-ISLR-Python/master/datasets/Boston.csv"
boston = pd.read_csv(path, index_col=0)
boston.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


### 3.6.2 Simple Linear Regression

#### Using scikit-learn

In [9]:
from sklearn import linear_model

# ols model with intercept
ols_sl = linear_model.LinearRegression(fit_intercept=True) 

x_train = np.asanyarray(boston['lstat']).reshape((-1, 1))
y_true = np.asanyarray(boston['medv'])
ols_sl.fit(x_train, y_true)

# summary
ols_sl.intercept_, ols_sl.coef_

(34.5538408793831, array([-0.95004935]))

In [10]:
# metrics
from sklearn.metrics import mean_squared_error

y_pred = ols_sl.predict(x_train)

ols_sl_summary = {'Ex. Var or R2_Score': ols_sl.score(x_train, y_true),
                  'MSE': mean_squared_error(y_true, y_pred)}

for k, v in ols_sl_summary.items():
    print(k, ':', v)

Ex. Var or R2_Score : 0.5441462975864797
MSE : 38.48296722989415


#### Using Statmodels

prints p-values, f-score, confidense intervals and ...

In [11]:
import statsmodels.api as sm

# predictor & dependent var
x_train = boston['lstat']
y_true = boston['medv']

# ols model with intercept added to predictor
ols_sm = sm.OLS(y_true, sm.add_constant(x_train)).fit()

# fitted model and summary
print(ols_sm.summary())

                            OLS Regression Results                            
Dep. Variable:                   medv   R-squared:                       0.544
Model:                            OLS   Adj. R-squared:                  0.543
Method:                 Least Squares   F-statistic:                     601.6
Date:                Fri, 25 Sep 2020   Prob (F-statistic):           5.08e-88
Time:                        05:58:25   Log-Likelihood:                -1641.5
No. Observations:                 506   AIC:                             3287.
Df Residuals:                     504   BIC:                             3295.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         34.5538      0.563     61.415      0.0

In [12]:
ols_sm.predict(sm.add_constant([5, 10, 15]))

array([29.80359411, 25.05334734, 20.30310057])

produce confidence and prediction interval for prediction of medv

In [27]:
predictions = ols_sm.get_prediction(sm.add_constant([5, 10, 15]))
intervals_df = predictions.summary_frame(alpha=0.05)
intervals_df.drop("mean_se", axis=1 , inplace=True)
intervals_df.columns = ["fit", "ci-lwr", "ci-upr", "pi-lwr", "pi-upr"]
intervals_df.head()

,fit,ci-lwr,ci-upr,pi-lwr,pi-upr
0,29.803594,29.007412,30.599776,17.565675,42.041513
1,25.053347,24.474132,25.632563,12.827626,37.279068
2,20.303101,19.731588,20.874613,8.077742,32.528459


In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot()
sns.regplot(x='lstat', y='medv', data=boston, ax=ax, line_kws={"linewidth":3, "color":'r'}, s=100)